In [ ]:
import os
import sys
import requests
from PIL import Image
from PIL.ExifTags import TAGS
from SPARQLWrapper import SPARQLWrapper, JSON

# Définition du point de terminaison Wikidata SPARQL
endpoint_url = "https://query.wikidata.org/sparql"

# Requête SPARQL pour récupérer les informations sur les tableaux
query = """
SELECT DISTINCT ?painting ?paintingLabel ?artistLabel ?genreLabel ?locationLabel ?date ?width ?height ?image WHERE {
  ?painting wdt:P31 wd:Q3305213.  # Sélectionne les peintures
  OPTIONAL { ?painting wdt:P170 ?artist. }  # Artiste
  OPTIONAL { ?painting wdt:P136 ?genre. }  # Genre artistique
  OPTIONAL { ?painting wdt:P276 ?location. }  # Lieu de conservation (musée)
  OPTIONAL { ?painting wdt:P571 ?date. }  # Date de création
  OPTIONAL { ?painting wdt:P2049 ?width. } # Largeur du tableau
  OPTIONAL { ?painting wdt:P2048 ?height. } # Hauteur du tableau
  OPTIONAL { ?painting wdt:P18 ?image. }  # Image du tableau

  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],fr,en". }
}
LIMIT 100
"""

# Fonction pour exécuter la requête SPARQL
def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (
        sys.version_info[0],
        sys.version_info[1],
    )
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

# Fonction pour récupérer les dimensions EXIF d'une image
def get_exif_data(image_url):
    try:
        # Télécharger l'image
        response = requests.get(image_url, stream=True)
        response.raise_for_status()

        # Ouvrir l'image avec PIL
        image = Image.open(response.raw)

        # Récupérer les dimensions
        width, height = image.size

        # Récupérer les métadonnées EXIF
        exif_data = image._getexif()
        exif_info = {}
        
        if exif_data:
            for tag, value in exif_data.items():
                tag_name = TAGS.get(tag, tag)
                exif_info[tag_name] = value

        # Déterminer l'orientation
        orientation = "Paysage (Horizontal)" if width > height else "Portrait (Vertical)"

        return {
            "ImageWidth": width,
            "ImageHeight": height,
            "Orientation": orientation,
            "EXIF_Metadata": exif_info
        }

    except Exception as e:
        return {"error": str(e)}

# Récupération des données Wikidata
array = []
results = get_results(endpoint_url, query)

for result in results["results"]["bindings"]:
    painting_name = result.get("paintingLabel", {}).get("value", "N/A")
    artist = result.get("artistLabel", {}).get("value", "N/A")
    genre = result.get("genreLabel", {}).get("value", "N/A")
    location = result.get("locationLabel", {}).get("value", "N/A")
    date = result.get("date", {}).get("value", "N/A")
    width = result.get("width", {}).get("value", "N/A")
    height = result.get("height", {}).get("value", "N/A")
    image_url = result.get("image", {}).get("value", "N/A")

    # Récupérer les métadonnées EXIF si une image est disponible
    exif_data = get_exif_data(image_url) if image_url != "N/A" else {}

    array.append(
        (
            painting_name,  # Nom du tableau
            artist,  # Nom de l'artiste
            genre,  # Genre artistique
            location,  # Lieu de conservation
            date,  # Date de création
            width,  # Largeur (Wikidata)
            height,  # Hauteur (Wikidata)
            image_url,  # URL de l'image
            exif_data.get("ImageWidth", "N/A"),  # Largeur EXIF
            exif_data.get("ImageHeight", "N/A"),  # Hauteur EXIF
            exif_data.get("Orientation", "N/A"),  # Orientation EXIF
            exif_data.get("EXIF_Metadata", {}),  # Métadonnées EXIF complètes
        )
    )

# Affichage des résultats
for item in array:
    print(item)


In [ ]:
import pandas as pd

# Création du DataFrame avec toutes les colonnes nécessaires
dataframe = pd.DataFrame(array, columns=[
    "tableau", "artiste", "genre", "lieu", "date",
    "largeur_wikidata", "hauteur_wikidata",
    "image", "largeur_exif", "hauteur_exif", "orientation_exif"
])

# Définition des types pour assurer un bon affichage
dataframe = dataframe.astype(
    dtype={
        "tableau": "<U200", "artiste": "<U200", "genre": "<U200",
        "lieu": "<U200", "date": "<U200",
        "largeur_wikidata": "<U20", "hauteur_wikidata": "<U20",
        "image": "<U300", "largeur_exif": "<U20", "hauteur_exif": "<U20", "orientation_exif": "<U50"
    }
)

# Affichage du DataFrame
dataframe
